# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [24]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


In [61]:
import yfinance as yf
import pandas as pd

fomc_statements = pd.read_csv('dates/fomc_statements.csv')
fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date']) 


ticker = '^IXIC'  
start_date = fomc_statements['Date'].min() - pd.Timedelta(days=15) 
end_date = fomc_statements['Date'].max() + pd.Timedelta(days=15)  

nasdaq_data = yf.download(ticker, start=start_date, end=end_date)

nasdaq_data.columns = nasdaq_data.columns.get_level_values(0)


nasdaq_data['return'] = nasdaq_data['Close'].pct_change()
nasdaq_data = nasdaq_data.dropna(subset=['return'])


rows = []

for date in fomc_statements['Date']:
    row = {'announcement_date': date, 'ticker': ticker}  
    for t in range(-10, 11): 
        target_date = date + pd.Timedelta(days=t)
        if target_date in nasdaq_data.index:  

            row[f'T{t:+}'] = nasdaq_data.loc[target_date, 'return']
        else:
            row[f'T{t:+}'] = pd.NA 
    rows.append(row)

final_df = pd.DataFrame(rows)

cols = ['ticker', 'announcement_date'] + [f'T{t:+}' for t in range(-10, 11)]
final_df = final_df[cols]

final_df
# final_df.to_csv('output.csv', index=False)  # if you want to save it


/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_89003/1368806922.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])
[*********************100%***********************]  1 of 1 completed


,ticker,announcement_date,T-10,T-9,T-8,T-7,T-6,T-5,T-4,T-3,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,^IXIC,2000-02-02,<NA>,-0.032894,0.017414,-0.023396,-0.007457,-0.037749,<NA>,<NA>,...,0.033633,0.007875,<NA>,<NA>,0.018291,0.024465,-0.014514,0.02805,-0.020104,<NA>
1,^IXIC,2000-03-21,<NA>,<NA>,-0.028004,-0.04088,-0.026348,0.029409,0.017115,<NA>,...,0.032487,0.015594,0.004538,<NA>,<NA>,-0.000901,-0.025142,-0.039145,-0.040214,0.025783
2,^IXIC,2000-05-16,<NA>,<NA>,-0.038629,-0.022993,-0.055866,0.033932,0.008424,<NA>,...,-0.019532,-0.02915,-0.041911,<NA>,<NA>,-0.007725,-0.059348,0.033515,-0.019953,-0.000075
3,^IXIC,2000-06-28,<NA>,0.033485,0.005898,0.01262,-0.031292,-0.023242,<NA>,<NA>,...,-0.016016,0.022924,<NA>,<NA>,0.00651,<NA>,-0.032273,0.025231,0.015813,<NA>
4,^IXIC,2000-08-22,<NA>,<NA>,0.015891,0.000512,0.002477,0.020634,-0.002672,<NA>,...,0.013339,0.010538,-0.002615,<NA>,<NA>,0.006904,0.002845,0.005301,0.024987,0.006652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,^IXIC,2024-09-18,<NA>,0.011609,0.008367,0.021711,0.010011,0.006506,<NA>,<NA>,...,0.025077,-0.003645,<NA>,<NA>,0.001446,0.005577,0.000426,0.005977,-0.003887,<NA>
202,^IXIC,2024-11-07,0.002623,0.00784,-0.005602,-0.027557,0.008000,<NA>,<NA>,-0.003286,...,0.000899,<NA>,<NA>,0.000621,-0.0009,-0.002628,-0.0064,-0.022375,<NA>,<NA>
203,^IXIC,2024-12-18,<NA>,-0.006197,-0.002505,0.017659,-0.006591,0.0012,<NA>,<NA>,...,-0.001027,0.010315,<NA>,<NA>,0.009824,0.013471,<NA>,-0.000538,-0.014901,<NA>
204,^IXIC,2025-01-29,<NA>,<NA>,0.006448,0.012783,0.002216,-0.004956,<NA>,<NA>,...,0.002518,-0.002759,<NA>,<NA>,-0.011997,0.013514,0.001949,0.005061,-0.013571,<NA>


In [58]:
import yfinance as yf
import pandas as pd

# Step 1: Read in the FOMC statement dates
fomc_statements = pd.read_csv('dates/fomc_statements.csv')
fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])  # ensure it's datetime format

# Step 2: Download NASDAQ index data from Yahoo Finance
ticker = '^IXIC'  # NASDAQ index symbol
start_date = fomc_statements['Date'].min() - pd.Timedelta(days=15)  # a little before the earliest FOMC date
end_date = fomc_statements['Date'].max() + pd.Timedelta(days=15)  # a little after the latest FOMC date

nasdaq_data = yf.download(ticker, start=start_date, end=end_date)

# Flatten the MultiIndex columns to use simple column names
nasdaq_data.columns = nasdaq_data.columns.get_level_values(0)

# Step 3: Calculate daily returns from the NASDAQ index's adjusted close price
nasdaq_data['return'] = nasdaq_data['Close'].pct_change()

# Step 4: Remove weekends and non-trading days
# Ensure we're only working with valid trading days
nasdaq_data = nasdaq_data[nasdaq_data.index.weekday < 5]  # Keep only weekdays (Monday to Friday)

# Step 5: Drop rows with NaN values in the 'return' column
nasdaq_data = nasdaq_data.dropna(subset=['return'])

# Step 6: Build the output dataframe with returns for T-10 to T+10 for each announcement date
rows = []

for date in fomc_statements['Date']:
    row = {'announcement_date': date, 'ticker': ticker}  # add ticker to each row
    valid_row = False  # Flag to check if the row contains any valid returns
    
    # Create a list to hold the return values from T-10 to T+10
    returns = []
    
    for t in range(-10, 11):  # from T-10 to T+10
        target_date = date + pd.Timedelta(days=t)
        
        if target_date in nasdaq_data.index:  # Check if the date is in the filtered data
            returns.append(nasdaq_data.loc[target_date, 'return'])
        else:
            returns.append(pd.NA)  # if missing (weekends or holidays)
    
    # Now we need to check if there are any NA values and shift accordingly
    for i in range(11):
        if pd.isna(returns[i]):
            if i < 10:  # If it's on the left side (T-10 to T-1)
                if pd.notna(returns[i + 1]):
                    returns[i] = returns[i + 1]  # Shift the value from the right side (T+1 to T+10)
                    returns[i + 1] = pd.NA  # Remove the value on the right (now moved to the left)
            if i > 0:  # If it's on the right side (T+1 to T+10)
                if pd.notna(returns[i - 1]):
                    returns[i] = returns[i - 1]  # Shift the value from the left side (T-10 to T-1)
                    returns[i - 1] = pd.NA  # Remove the value on the left (now moved to the right)
    
    # Add the cleaned return values to the row
    for t in range(-10, 11):
        row[f'T{t:+}'] = returns[t + 10]  # Adjust index to match T-10 to T+10 format
    
    rows.append(row)

final_df = pd.DataFrame(rows)

# Step 7: Optional - reorder the columns nicely
cols = ['ticker', 'announcement_date'] + [f'T{t:+}' for t in range(-10, 11)]
final_df = final_df[cols]

# Step 8: Print or export the result
final_df
# final_df.to_csv('output.csv', index=False)  # if you want to save it


/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_89003/3347005618.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])  # ensure it's datetime format
[*********************100%***********************]  1 of 1 completed


,ticker,announcement_date,T-10,T-9,T-8,T-7,T-6,T-5,T-4,T-3,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,^IXIC,2000-02-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.033633,0.007875,<NA>,<NA>,0.018291,0.024465,-0.014514,0.02805,-0.020104,<NA>
1,^IXIC,2000-03-21,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.032487,0.015594,0.004538,<NA>,<NA>,-0.000901,-0.025142,-0.039145,-0.040214,0.025783
2,^IXIC,2000-05-16,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,-0.019532,-0.02915,-0.041911,<NA>,<NA>,-0.007725,-0.059348,0.033515,-0.019953,-0.000075
3,^IXIC,2000-06-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,-0.016016,0.022924,<NA>,<NA>,0.00651,<NA>,-0.032273,0.025231,0.015813,<NA>
4,^IXIC,2000-08-22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.013339,0.010538,-0.002615,<NA>,<NA>,0.006904,0.002845,0.005301,0.024987,0.006652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,^IXIC,2024-09-18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.025077,-0.003645,<NA>,<NA>,0.001446,0.005577,0.000426,0.005977,-0.003887,<NA>
202,^IXIC,2024-11-07,0.002623,0.00784,-0.005602,-0.027557,<NA>,<NA>,<NA>,<NA>,...,0.000899,<NA>,<NA>,0.000621,-0.0009,-0.002628,-0.0064,-0.022375,<NA>,<NA>
203,^IXIC,2024-12-18,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,-0.001027,0.010315,<NA>,<NA>,0.009824,0.013471,<NA>,-0.000538,-0.014901,<NA>
204,^IXIC,2025-01-29,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,0.002518,-0.002759,<NA>,<NA>,-0.011997,0.013514,0.001949,0.005061,-0.013571,<NA>
